In [1]:
from dotenv import load_dotenv

assert load_dotenv("../.env")

In [2]:
import ragna

ragna.__version__

'0.1.dev17+gfcf7439'

In [3]:
from ragna.core import Config

config = Config()
config

Config(local_cache_root=PosixPath('/home/philip/.cache/ragna'), state_database_url='sqlite:////home/philip/.cache/ragna/ragna.db', queue_database_url='redis://127.0.0.1:6379', registered_source_storage_classes={}, registered_llm_classes={})

In [4]:
document_path = "demo_document.txt"

with open(document_path, "w") as file:
    file.write("Ragna is an open-source RAG orchestration app\n")

In [5]:
from ragna.core import Rag
from ragna.llm import RagnaDemoLlm
from ragna.source_storage import RagnaDemoSourceStorage

rag = Rag(config, start_ragna_worker=True)

chat = await rag.start_new_chat(
    documents=[document_path],
    source_storage=RagnaDemoSourceStorage,
    llm=RagnaDemoLlm,
)

prompt = "What is Ragna?"
await chat.answer(prompt)

20:22:34 Worker rq:worker:6f9db5f2392d4a8c9fd2684966f53ea5 started with PID 47562, version 1.15.1
20:22:34 Subscribing to channel rq:pubsub:6f9db5f2392d4a8c9fd2684966f53ea5
20:22:34 *** Listening on default...
20:22:34 Cleaning registries for queue: default
{"event": "No registered components available", "timestamp": "2023-09-11T18:22:36.697954Z", "level": "warning", "pathname": "/home/philip/git/ora/ragna/core/_rag.py", "lineno": 98}
{"event": "No registered components available", "timestamp": "2023-09-11T18:22:36.777673Z", "level": "warning", "pathname": "/home/philip/git/ora/ragna/core/_rag.py", "lineno": 98}
20:22:36 default: ragna.core._queue.Worker._execute_job(b"\x80\x05\x95\t\x0b\x00\x00\x00\x00\x00\x00\x8c\x17cloudpickle.cloudpickle...) (dddafab3-d374-4b75-9ccd-74979180079d)
20:22:36 default: Job OK (dddafab3-d374-4b75-9ccd-74979180079d)
20:22:36 Result is kept for 500 seconds
20:22:37 default: ragna.core._queue.Worker._execute_job(b"\x80\x05\x95e\n\x00\x00\x00\x00\x00\x00\x8c

"I'm just pretending to be an LLM, so I can't actually help with your prompt."

In [6]:
import itertools
import asyncio
from pprint import pprint

from ragna.llm import OpenaiGpt35Turbo16kLlm, OpenaiGpt4Llm
from ragna.source_storage import ChromaSourceStorage

source_storages = [ChromaSourceStorage]
llms = [OpenaiGpt35Turbo16kLlm, OpenaiGpt4Llm]


async def answer_prompt(source_storage, llm):
    chat = await rag.start_new_chat(
        documents=[document_path],
        source_storage=source_storage,
        llm=llm,
    )
    return await chat.answer(prompt)


experiments = {
    (source_storage.display_name(), llm.display_name()): answer_prompt(
        source_storage, llm
    )
    for source_storage, llm in itertools.product(source_storages, llms)
}

pprint(experiments)

{('Chroma', 'OpenAI/gpt-3.5-turbo-16k'): <coroutine object answer_prompt at 0x7f81e2d33c40>,
 ('Chroma', 'OpenAI/gpt-4'): <coroutine object answer_prompt at 0x7f81e2d33cc0>}


In [7]:
results = dict(zip(experiments.keys(), await asyncio.gather(*experiments.values())))

pprint(results)

20:22:38 default: ragna.core._queue.Worker._execute_job(b"\x80\x05\x95\xe8\n\x00\x00\x00\x00\x00\x00\x8c\x17cloudpickle.cloudpickle...) (ff34f99d-6795-44c3-ba9a-cc435ac94610)
20:22:38 default: Job OK (ff34f99d-6795-44c3-ba9a-cc435ac94610)
20:22:38 Result is kept for 500 seconds
20:22:38 default: ragna.core._queue.Worker._execute_job(b"\x80\x05\x95\xe8\n\x00\x00\x00\x00\x00\x00\x8c\x17cloudpickle.cloudpickle...) (07ce88b5-73b3-47dc-bac4-6134e000a6ed)
20:22:39 default: Job OK (07ce88b5-73b3-47dc-bac4-6134e000a6ed)
20:22:39 Result is kept for 500 seconds
20:22:39 default: ragna.core._queue.Worker._execute_job(b"\x80\x05\x95B\n\x00\x00\x00\x00\x00\x00\x8c\x17cloudpickle.cloudpickle\x9...) (4f534fde-3a2d-4942-be0f-8eab2fa10b7a)
20:22:39 default: Job OK (4f534fde-3a2d-4942-be0f-8eab2fa10b7a)
20:22:39 Result is kept for 500 seconds
20:22:39 default: ragna.core._queue.Worker._execute_job(b"\x80\x05\x95B\n\x00\x00\x00\x00\x00\x00\x8c\x17cloudpickle.cloudpickle\x9...) (a438f35a-3cfa-4e9a-ac3a-64